# Multi-armed bandit


In [1]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None

## Baseline: e-greedy

сначала прогоним код бейзлайна, который нам был дан, чтобы было с чем сравнивать

In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [3]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
print(end - start)

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
218.08477687835693


In [4]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [5]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


## Upper-confidence bound (UCB)

теперь напишем алгоритм UCB, по дефолту множитель перед exploration частью будет равен 1, попробуем его потюнить

In [6]:
from math import log

def UCB(history: pd.DataFrame, explor_coef: float = 1.0):
    # t - сколько всего раз дергали за все ручки (показывали какой-либо баннер)
    t = history['impressions'].sum()
    # Берем верхнюю квантиль у каждой ручки
    ub = history['clicks'] / (history['impressions'] + 1) + explor_coef * np.sqrt(2 * log(t + 1) / (history['impressions'] + 1))
    # Выбираем ручку с максимальной квантилью
    n = np.argmax(ub)
    return history.index[n]

попробуем разные значения множителя перед exploration частью, затем выберем лучший результат на данной симуляции

In [7]:
explore_coeffs = [0.001, 0.01, 0.1, 1, 10, 100]
results = []

for coef in explore_coeffs:
    policy = partial(UCB, explor_coef = coef)
    seed = 18475
    np.random.seed(seed=seed)
    output = simulation(policy, n=200000, seed=seed)
    r, h = output['regret'], output['history']
    results.append([coef, r, h])

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impr

In [8]:
res_df = pd.DataFrame([x[:2] for x in results], columns=['coef', 'regret'])
res_df

,coef,regret
0,0.001,7570.250349
1,0.010,8430.422548
2,0.100,211.362165
3,1.000,10457.518552
4,10.000,27948.355557
5,100.000,29215.148043


In [9]:
# output['history'] для лучшего результата
results[np.argmin(res_df['regret'])][2]

,impressions,clicks,lifetime,p
153,4.0,0.0,18003.025431,0.220134
162,4.0,0.0,1537.166719,0.113780
172,21177.0,4621.0,19648.592394,0.219968
173,4.0,0.0,12771.474990,0.122694
180,4.0,0.0,4655.819793,0.020061
182,4.0,0.0,889.624649,0.004621
183,4.0,0.0,15187.163761,0.073886


Из всех коэффициентов при множителе exploration лучшим оказался 0.1 с результатом regret=211.

## Thompson sampling Бернулли бандит

теперь попробуем третий вариант алгоритма, рассмотренного на лекции.

In [10]:
def TS(history: pd.DataFrame):
    # считаем альфы для каждой ручки - кол-во успехов + 1 (у нас - клики + 1)
    alphas = history['clicks'] + 1
    # считаем беты для каждой ручки - кол-во неуспехов (у нас - показы-клики+1)
    betas = history['impressions'] - history['clicks'] + 1
    # для каждой ручки сэмплируем вероятность успеха из бета-распределения
    probs = np.random.beta(alphas, betas, size=len(history))
    # берем ручку с максимальной сэмплированной вероятностью
    n = np.argmax(probs)
    return history.index[n]

запустим его на нашей симуляции

In [11]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

output = simulation(TS, n=200000, seed=seed)

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


In [12]:
# TS regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1223.6103636831504, 0.006118051818415752, 184)

In [13]:
output['history']

,impressions,clicks,lifetime,p
153,24135.0,5317.0,18003.025431,0.220134
162,113.0,12.0,1537.166719,0.113780
172,3521.0,734.0,19648.592394,0.219968
173,150.0,17.0,12771.474990,0.122694
180,31.0,1.0,4655.819793,0.020061
182,19.0,0.0,889.624649,0.004621
183,26.0,1.0,15187.163761,0.073886


## Выводы

Полученные значения regret для всех алгоритмов:

+ Baseline (epsilon-greedy): 1541
+ UCB (множитель exploration=0.1): 211
+ TS: 1224

Видно, что UCB с тюнингом множителя у exploration части и TS оказались лучше, чем бейзлайноый epsilon-greedy.